In [1]:
import os, sys
import time
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator
import heapq as hq
import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [ ]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

In [ ]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

# Prepare models/datasets
#### Load the pretrained models "lstm" (entire_model_1m_20interactions.pt) and "pooling" (pooling_model_1m_20interactions.pt)

In [ ]:
# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire')
pooling_model = load_model('pooling')

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

#### Get the dataset Movielens with the variant 1M. Then divide it into a training set and a testing set. And finally this code limits the length of each sequence of elements in the 2 sets to 20.

In [ ]:
from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset 
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length)

#### 1. Compute the similarity matrix based on cosine similarity
#### 2. Compute the similarity matrix based on Jaccard similarity

In [ ]:
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

# Various implemented Strategies

In [ ]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget,  model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10
        self.budget = init_budget

    #Must be implemented by subclasses. Used to select the next item to recommand to the user.
    def next_comb(self, reverse=False):
        raise NotImplementedError

    
    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    #Method to reset the costs of the last recommended combination
    def reset_costs(self):
        self.last_comb_cost = 0

    #Returns the initial budget
    def get_init_budget(self):
        return self.budget

### RandomSelection
#### This class is a subclass of the "BaseStrategy" class, representing a random item selection strategy for the sequential recommendation task.